In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch
import json
from datetime import datetime

In [ ]:
es = Elasticsearch(timeout=5000)

page = es.search(index='jaeger-span-2022-05-19', scroll='10m', size=10000, body={"query": {
  "match": {
         "kind":"SERVER"
             }
      },
  "query":{
    "bool": {
      "must": [{
        "script": {
          "script": {
            "inline": "doc['traceID.keyword'].value == doc['spanID.keyword'].value"

                  }
                } 
              }]
            }
          }})

query_request = []

sid = page['_scroll_id']
print(page['hits']['total']['value'])

while(int(page['hits']['total']['value']) > 0):
    print("Scrolling...", str(int(page['hits']['total']['value'])))
    query_request.append(page['hits']['hits'])
    page = es.scroll(scroll_id=sid, scroll='10m')
    page['hits']['total']['value'] = len(page['hits']['hits'])


print(len(query_request))

result = []

for i in query_request:
    for j in i:
        result.append(j['_source'])

print(len(result))

In [ ]:
df= pd.DataFrame(result)
df

In [ ]:
for i in df['timestamp'].index:
    df.loc[i,'timestamp'] = datetime.utcfromtimestamp(df.loc[i,'timestamp']/1000000)
df    

In [ ]:
timestamp1 = 1652979531
timestamp2 = 1652983145
start = datetime.fromtimestamp(timestamp1)
end = datetime.fromtimestamp(timestamp2)
print(start,end)

In [ ]:
mask = (df['timestamp'] > start) & (df['timestamp'] <= end)
mask

In [ ]:
filtered_df=df.loc[mask]
print(filtered_df)

In [ ]:
import matplotlib.pyplot as plt
# set up figure & axes
fig, axes = plt.subplots(figsize=(20,20))

# drop sharex, sharey, layout & add ax=axes
df.hist(column='duration',by='name', ax=axes)

# set title and axis labels
fig.text(0.01, 0.01, '')
fig.text(0.01, 0.01, '', rotation='vertical')